In [ ]:
import pandas as pd
import numpy as np
import math
import time
from pandas.api.types import CategoricalDtype
import gspread
import os

In [ ]:
# key parameters

max_year = 2023
min_year = 2021

client_secret = "Desktop/GEM_INFO/client_secret.json"
client_secret_full_path = os.path.expanduser("~/") + client_secret

gogpt_dash_file_h1_2023 = 'Global Oil and Gas Plant Tracker (GOGPT) compiled 2023-08-18.xlsx'
gogpt_dash_file_h2_2022 = 'Global Gas Plant Tracker (GGPT) 2023-02.xlsx'
gogpt_dash_file_h1_2022 = 'Global Gas Plant Tracker 2022-08.xlsx'
gogpt_dash_file_h2_2021 = 'Global Gas Plant Tracker (GGPT) 2022-02.xlsx'
gogpt_dash_file_h1_2021 = 'Global Gas Plant Tracker 2021-10.xlsx'

gogpt_dash_path = os.path.expanduser('/Users/gem-tah/Desktop/GEM_INFO/GEM_WORK/GOGPT/GOGPT-Dashboard/data/pre-2023-08/')


In [ ]:
gogpt_xl_h1_2023 = pd.ExcelFile(gogpt_dash_path + gogpt_dash_file_h1_2023)
gogpt_xl_h2_2022 = pd.ExcelFile(gogpt_dash_path + gogpt_dash_file_h2_2022)
gogpt_xl_h1_2022 = pd.ExcelFile(gogpt_dash_path + gogpt_dash_file_h1_2022)
gogpt_xl_h2_2021 = pd.ExcelFile(gogpt_dash_path + gogpt_dash_file_h2_2021)
gogpt_xl_h1_2021 = pd.ExcelFile(gogpt_dash_path + gogpt_dash_file_h1_2021)



In [ ]:
def gspread_access_file_read_only(key, title):
    """
    key = Google Sheets unique key in the URL
    title = name of the sheet you want to read
    """
    gspread_creds = gspread.oauth(
        scopes=["https://www.googleapis.com/auth/spreadsheets.readonly"],
        credentials_filename=client_secret_full_path,
        # authorized_user_filename=json_token_name,
    )
    gsheets = gspread_creds.open_by_key(key)
    # Access a specific tab
    spreadsheet = gsheets.worksheet(title)
    # expected_header option provided following: https://github.com/burnash/gspread/issues/1007
    df = pd.DataFrame(spreadsheet.get_all_records(expected_headers=[]))
    
    return df

def determine_year_month_from_file(gogpt_dash_file):
    yyyy_mm = gogpt_dash_file.split('.xlsx')[0].split(' ')[-1]
    month_int = int(yyyy_mm.split('-')[1])
    year_int = int(yyyy_mm.split('-')[0])

    print(f"year_int: {year_int} month_int: {month_int}")

    return month_int, year_int

In [ ]:
# create list of countries to choose from (GEM country names)
# data in gogpt_status is most complete; example of Albania, which only has cancelled units, nothing else

list_of_xls_paths = [gogpt_xl_h1_2023, gogpt_xl_h2_2022, gogpt_xl_h1_2022, gogpt_xl_h2_2021, gogpt_xl_h1_2021]

month_h1_2023, year_h1_2023 = determine_year_month_from_file(gogpt_dash_file_h1_2023)
month_h2_2022, year_h2_2022 = determine_year_month_from_file(gogpt_dash_file_h2_2022)
month_h1_2022, year_h1_2022 = determine_year_month_from_file(gogpt_dash_file_h1_2022)
month_h2_2021, year_h2_2021 = determine_year_month_from_file(gogpt_dash_file_h2_2021)
month_h1_2021, year_h1_2021 = determine_year_month_from_file(gogpt_dash_file_h1_2021)

gogpt_h1_2023 = pd.read_excel(gogpt_xl_h1_2023, sheet_name = 'Gas &amp; Oil Units')
gogpt_h1_2023['Version month'] = month_h1_2023
gogpt_h1_2023['Version year'] = year_h1_2023

gogpt_h2_2022 = pd.read_excel(gogpt_xl_h2_2022, sheet_name = 'Gas Units')
gogpt_h2_2022['Version month'] = month_h2_2022
gogpt_h2_2022['Version year'] = year_h2_2022

gogpt_h1_2022 = pd.read_excel(gogpt_xl_h1_2022, sheet_name = 'Gas plants - data')
gogpt_h1_2022['Version month'] = month_h1_2022
gogpt_h1_2022['Version year'] = year_h1_2022

gogpt_h2_2021 = pd.read_excel(gogpt_xl_h2_2021, sheet_name = 'Gas Units')
gogpt_h2_2021['Version month'] = month_h2_2021
gogpt_h2_2021['Version year'] = year_h2_2021

gogpt_h1_2021 = pd.read_excel(gogpt_xl_h1_2021, sheet_name = 'GGPT - Gas Units')
gogpt_h1_2021['Version month'] = month_h1_2021
gogpt_h1_2021['Version year'] = year_h1_2021

gogpt_country_list = gogpt_h1_2023['Country'].sort_values().unique().tolist()
gogpt_country_list = ['all'] + gogpt_country_list

In [ ]:
# drop rows where capacity or country is na or not found


list_of_dfs = [gogpt_h1_2023, gogpt_h2_2022, gogpt_h1_2022, gogpt_h2_2021, gogpt_h1_2021]

def clean_status_dfs(df_list):
    list_of_dfs_clean = []
    for df in list_of_dfs:
        print(df.shape)
        # TODO test to see if this is removing more than we want
        df = df.rename(columns={'Capacity elec. (MW)': 'Capacity (MW)'})
        # df = df.dropna(subset=['Capacity (MW)', 'Country']) # capacity null items in gogpt_h1_2022 
        df = df.replace({'Capacity (MW)':'not found'}, 0)
        # df = df.drop(df[df['Capacity (MW)'] == 'not found'].index) # hits all but gogpt_h1_2022
        print(df.shape)
        list_of_dfs_clean.append(df)
    return list_of_dfs_clean

gogpt_h1_2023, gogpt_h2_2022, gogpt_h1_2022, gogpt_h2_2021, gogpt_h1_2021 = clean_status_dfs(list_of_dfs)




### Map (upper left)
- pivot capacity by country in gogpt_h1_2023 to get map tab for gogpt input


In [ ]:
# create pivot on capacities from country
# gogpt_h1_2023 is df, so pivot sum on column ['Capacity MW'] for all ['Country'] 
# remove unnecessary columns
# then groupby country and sum


gogpt_map = gogpt_h1_2023[['Country', 'Capacity (MW)', 'Status']].copy() 
print(gogpt_map.shape[0])
# filter by operating status then sum capacities by country
# TODO: wasn't filtering for operating that before so check GCPT! 
gogpt_map = gogpt_map[gogpt_map['Status'] == 'operating'] 
print(gogpt_map.shape[0])

gogpt_map['Capacity (MW)'] = pd.to_numeric(gogpt_map['Capacity (MW)'])


# gogpt_map = gogpt_map.groupby(['Country']).sum().reset_index()
# gogpt_map = gogogpt_map = gogpt_map.groupby(['Country']).sum().reset_index()
gogpt_map = gogpt_map.groupby(['Country'])[['Capacity (MW)']].sum().reset_index()

# gogpt_map = gogpt_map.groupby(by="Country")["Capacity (MW)"].sum()
# Convert "to int"
gogpt_map = gogpt_map.astype({'Capacity (MW)':'int'})
# rslt_df_op[rslt_df_op['Country'] == 'Algeria'] # exact
# rslt_df_op[rslt_df_op['Country'] == 'Brazil'] # exact
# rslt_df_op[rslt_df_op['Country'] == 'China'] # exact
# print(rslt_df_op['Capacity (MW)'].sum()) # slightly off compared to pivot probably float v int known issue

In [ ]:
def create_country_code_df():

    gem_naming_convention_key = '1mtlwSJfWy1gbIwXVgpP3d6CcUEWo2OM0IvPD6yztGXI'

    # gspread way
    country_codes = gspread_access_file_read_only(gem_naming_convention_key, 'Countries')

    # clean up the codes to remove non-printing characters from wikipedia
    for col in country_codes.columns:
        if country_codes[col].dtype == object:
            country_codes[col] = country_codes[col].str.replace('\xa0', '', regex=False)

    # get rid of parenthetical footnotes at end of names & whitespace
    country_codes['ISO 3166 Country Name'] = country_codes['ISO 3166 Country Name'].str.split('[').str[0].str.strip()

    return country_codes

In [ ]:
def create_country_name_conversions_dict():
    """
    Creates a dictionary for converting GEM standard country names to ISO 3166 versions.
    
    This downloads the GEM standard names file from Google Sheets using pygsheets,
    then pares down the df to only those in which GEM uses a different name than ISO 3166.

    Then it creates a dictionary, which can be used for converting from GEM to ISO.
    
    This is needed because Plotly's choropleth function uses ISO names for getting the outline for each country.
    """
    
    standard_country_names_key = '1mtlwSJfWy1gbIwXVgpP3d6CcUEWo2OM0IvPD6yztGXI'

    # gspread way
    df = gspread_access_file_read_only(standard_country_names_key, 'Countries')
    # keep only those with a mismatch
    name_diffs = df.copy()
    name_diffs = name_diffs[name_diffs['GEM name same as ISO 3166?']=='FALSE']
    # exclude those not in ISO
    name_diffs = name_diffs[name_diffs['ISO 3166 Country Name']!='NOT LISTED']
    name_diffs_dict = name_diffs.set_index('GEM Standard Country Name')['ISO 3166 Country Name'].to_dict()   
    
    return name_diffs_dict

In [ ]:

# add any missing countries, to make sure that all countries in gogpt_country_list are in gogpt_map
missing_countries = [x for x in gogpt_country_list if x not in gogpt_map['Country'].tolist()]
if len(missing_countries) > 0:
    print(f"These countries were missing: {missing_countries}")
    missing_df = pd.DataFrame.from_dict({
            'Country': missing_countries, 
            'Capacity (MW)': [float(0)]*len(missing_countries)
        }, orient='columns')
    gogpt_map = pd.concat([gogpt_map, missing_df], sort=False)

# change GEM country names to ISO 3166
name_diffs_dict = create_country_name_conversions_dict()
gogpt_map['ISO 3166 Country Name'] = gogpt_map['Country'].replace(name_diffs_dict)

# Note: Kosovo isn't recognized in ISO 3166, so can't be shown on Plotly map on its own. 
# We could combine it with Serbia for display--but then our data in the dashboard wouldn't be the same as in our spreadsheets & other maps.
# Wikipedia said Kosovo declared independence from Serbia in 2008; it is only partially recognized.

# show countries in gogpt_map not in gogpt_country_list:
extraneous_countries = [x for x in gogpt_map['Country'].tolist() if x not in gogpt_country_list]
if len(extraneous_countries) > 0:
    print(f"Extraneous countries to be removed: {extraneous_countries}")

# keep only countries that are in gogpt_country_list
gogpt_map = gogpt_map[gogpt_map['Country'].isin(gogpt_country_list)]

# merge in ISO country codes (needed by Plotly)
country_codes = create_country_code_df()
gogpt_map = pd.merge(
    country_codes[['ISO 3166 Country Name', 'Country ISO 3166-1 alpha-3']],
    gogpt_map,
    on='ISO 3166 Country Name', 
    how='outer'
)
gogpt_map = gogpt_map.rename(columns={'Country ISO 3166-1 alpha-3': 'iso_alpha'})

# exclude those with no value for iso_alpha
# This excludes notes within the ISO dataset, e.g., "Akrotiri and Dhekelia – See United Kingdom, The."
# Unfortunately, this also excludes Kosovo from gogpt
gogpt_map = gogpt_map[gogpt_map['iso_alpha'].isna()==False]

In [ ]:
# if capacity is 0, instead use 1, to avoid zero capacity leading to -inf log value
# similar approach is used in numpy log1p
# https://numpy.org/doc/stable/reference/generated/numpy.log1p.html
gogpt_map['capacity log10 + 1'] = np.log10(gogpt_map['Capacity (MW)'].replace(float(0), float(1)))

# create hover text
# (note: variable hover_text below is a Pandas Series)
hover_text = gogpt_map['Country'] + ': '
hover_text = hover_text + gogpt_map['Capacity (MW)'].map('{:,.0f}'.format) + ' MW'
# hide extra bit, e.g. 'trace 0'; based on https://plotly.com/python/reference/#scatter-hovertemplate
hover_text = hover_text + '<extra></extra>'
gogpt_map['hover_text'] = hover_text

In [ ]:
gogpt_map  

### Status graph
- from orig get status column, and country column, and capacity column groupby country and status, removing retired and cancelled
- do this for 2020, 2021, 2022, 2023 datasets and create column for which year it is then combine all dfs

In [ ]:
list_of_dfs_status = [gogpt_h1_2023, gogpt_h2_2022, gogpt_h1_2022, gogpt_h2_2021, gogpt_h1_2021]
accepted_statuses = ['announced', 'construction', 'mothballed', 'operating', 'pre-construction', 'shelved']


def filter_by_accepted_status(list_of_dfs):
    filtered_status_dfs= []
    for df in list_of_dfs:
        df_status = df[['Country', 'Capacity (MW)', 'Status', 'Version month', 'Version year', 'GEM unit ID']].copy() 
        # before = df_status.shape[0]
        # print(f'before: {df_status.shape[0]}')
        df_status['Status'] = df_status['Status'].replace({'proposed': 'pre-construction'})
        df_status = df_status[df_status['Status'].isin(accepted_statuses)]
        # after = df_status.shape[0]
        # print(f'after: {df_status.shape[0]}')
        # if before - after == unwanted:
        #     print("all accounted for")
        # else:
        #     print("could be a slight problem, numbers don't line up")

        version_month = df_status['Version month'].values[0]
        version_year = df_status['Version year'].values[0]
        df_status = df_status[['Country', 'Capacity (MW)', 'Status']].copy()
        
        table = pd.pivot_table(df_status, values='Capacity (MW)', index=['Country','Status'], aggfunc='sum', fill_value=0).reset_index()
        print(table[table['Country'] == 'China'])
        # TODO remove below groupby
        # df_status_mw = df_status_mw.groupby(['Country','Status', 'Capacity (MW)']).sum(['Capacity (MW)']).reset_index()
        # df_status_mw = df_status_mw.groupby(['Country','Status']).sum(['Capacity (MW)']).reset_index()

        # df_status = df_status.rename(columns={
        # 'Status': version_year, 
        # })
        if version_month >= 6:
            xaxis_version_year = version_year + .5
        else:
            xaxis_version_year = version_year
        status_column = f'{xaxis_version_year}'
        table = table.rename(columns={
            'Status': status_column, 
            })
        filtered_status_dfs.append(table)

    return filtered_status_dfs

status_gogpt_h1_2023, status_gogpt_h2_2022, status_gogpt_h1_2022, status_gogpt_h2_2021, status_gogpt_h1_2021 = filter_by_accepted_status(list_of_dfs_status)


In [ ]:
# tests against excel
status_gogpt_h1_2023[status_gogpt_h1_2023['Country'] == 'Brazil']
status_gogpt_h1_2023[status_gogpt_h1_2023['Country'] == 'China']
status_gogpt_h1_2023[status_gogpt_h1_2023['Country'] == 'Algeria']


test that the appropriate rows were removed via the status function above

In [ ]:
def filter_by_not_accepted_status(list_of_dfs):
    re_filtered_status_dfs = []
    for df in list_of_dfs:
        df_status = df[['Country', 'Capacity (MW)', 'Status', 'Version month', 'Version year', 'GEM unit ID']].copy() 

        df_status['Status'] = df_status['Status'].replace({'proposed': 'pre-construction'})
        df_status = df_status[~df_status['Status'].isin(accepted_statuses)]
        # sum the capacity for all plants that have a bad status to see how much we'll miss in case things don't line up
        version_month = df_status['Version month'].values[0]
        version_year = df_status['Version year'].values[0]
        df_status = df_status[['Country', 'Capacity (MW)', 'Status']].copy()
        # df_status = df_status.groupby(['Country','Status', 'Capacity (MW)']).sum(['Capacity (MW)']).reset_index()
        
        df_status_mw = df_status.groupby(['Country','Status','Capacity (MW)']).sum(['Capacity (MW)']).reset_index()
        df_status_mw = df_status_mw.groupby(['Country','Status']).sum(['Capacity (MW)']).reset_index()
        
        if version_month >= 6:
            xaxis_version_year = version_year + .5
        else:
            xaxis_version_year = version_year
        status_column = f'{xaxis_version_year}'
        df_status_mw = df_status_mw.rename(columns={
            'Status': status_column, 
            })
        # df_status['xaxis'] = xaxis_version_year
        re_filtered_status_dfs.append(df_status_mw)

    return re_filtered_status_dfs

restatus_gogpt_h1_2023, restatus_gogpt_h2_2022, restatus_gogpt_h1_2022, restatus_gogpt_h2_2021, restatus_gogpt_h1_2021 = filter_by_not_accepted_status(list_of_dfs_status)


restatus_gogpt_h1_2023
print(restatus_gogpt_h1_2023[restatus_gogpt_h1_2023['Country'] == 'Brazil'])
#    Country     2023.5  Capacity (MW)
# 16  Brazil  cancelled          260.0
# 17  Brazil  cancelled         1768.0
restatus_gogpt_h1_2023[restatus_gogpt_h1_2023['Country'] == 'Brazil']
restatus_gogpt_h1_2023
countries_unique_check = restatus_gogpt_h1_2023['Country'].unique()
print(sorted(countries_unique_check))
print(len(countries_unique_check))


In [ ]:
#merge all status dfs


gogpt_status_merged = pd.merge(
    status_gogpt_h1_2023,
    status_gogpt_h2_2022,
    on=['Country', 'Capacity (MW)'],
    how='outer'
)

gogpt_status_merged = pd.merge(
    gogpt_status_merged,
    status_gogpt_h1_2022,
    on=['Country', 'Capacity (MW)'],
    how='outer'
)

gogpt_status_merged = pd.merge(
    gogpt_status_merged,
    status_gogpt_h2_2021,
    on=['Country', 'Capacity (MW)'],
    how='outer'
)

gogpt_status_merged = pd.merge(
    gogpt_status_merged,
    status_gogpt_h1_2021,
    on=['Country', 'Capacity (MW)'],
    how='outer'
)

In [ ]:
def status_clean_data(df):
    for col in df.columns:
        if col not in ['Country', 'Capacity (MW)', 'xaxis']:
            # make the text lowercase
            print(f"Cleaning col {col}")
            df[col] = df[col].str.lower()

    return df

In [ ]:
def sort_status(df):
    """
    convert column 'Status' to categorical
    https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html
    """
    
    status_order = [
        'operating',
        'mothballed',
        'announced',
        'pre-construction',
        'construction',
        # 'retired',
        'shelved',
        # 'cancelled',
    ]
    df['Status'] = df['Status'].astype(
        CategoricalDtype(status_order, ordered=False)
    )    
    df = df.sort_values(by=['Country', 'Status', 'Year'])
    
    return df

In [ ]:
def status_test_capacity(df, df_init):
    """Runs at end of status_condense, to check that there was no change in the data.
    (Comparing version after condensing the data as it was before condensing the data.)
    
    There are entries with status NaN in the version of the data before condensing,
    because the table prior to that step has a row for each unit, and columns for each year.
    (Other distinguishing features for each unit aren't included in this table.)
    
    If there is no data for a unit for some of the years covered, then we have status NaN for those years.
    (Example: It was proposed only in 2018, then the years 2014-2017 have status NaN.)
    """

    df = df.reset_index()
        
    all_statuses = []

    for year in ['2021.5', '2022', '2022.5', '2023', '2023.5']:
        all_statuses += df_init[year].tolist() 

    all_statuses = list(set(all_statuses))

    for sel_status in all_statuses:
        print(f"Testing capacities for {sel_status}")
        for sel_year in ['2021.5', '2022', '2022.5', '2023', '2023.5']:
            sel_init = df_init[df_init[sel_year]==sel_status]
            sel_init_sum = sel_init['Capacity (MW)'].sum()

            # handle two different formats for df
            if 'Status' in df.columns:
                sel = df[(df['Year']==sel_year) & (df['Status']==sel_status)]
                sel_sum = sel['Capacity (MW)'].sum()
                
            else:
                sel = df[df[sel_year]==sel_status]
                sel_sum = sel['Capacity (MW)'].sum()

            # compare values
            abs_diff = abs(sel_sum - sel_init_sum)
            if abs_diff <= 1e-7:
                pass

            elif abs_diff > 1e-7:
                print("Error!" + f" Capacity difference for {sel_year} & {sel_status}.")
                print(f"Initial value: {sel_init_sum}")
                print(f"Current value: {sel_sum}")
            else:
                print("Unexpected case")
    
    # no return

In [ ]:
def status_fill_in_missing(df):
    """
    For each country, for each status, fill in zero value if it is missing.
    """
    # year_range = range(df['Year'].min(), df['Year'].max() + 1)
    year_range = ['2021.5', '2022', '2022.5', '2023', '2023.5']
    statuses = df['Status'].dropna().unique().tolist()
    for country in df['Country'].unique().tolist():
        for status in statuses:
            for year in year_range:
                df_sel = df[(df['Year']==year) & (df['Country']==country) & (df['Status']==status)]
                if len(df_sel) == 0:
                    # there is a missing value; fill it in
                    # print(f"To fill in missing value for {country}, {year}, {status}") # for db
                    # values in dict below must be lists, to avoid error:
                    # "ValueError: If using all scalar values, you must pass an index"
                    new_df = pd.DataFrame.from_dict({
                                'Country': [country],
                                'Year': [year],
                                'Status': [status],
                                'Capacity (MW)': [float(0)],
                            }, orient='columns')
                    df = pd.concat([df, new_df], sort=False)

    df = df.sort_values(by=['Country', 'Year', 'Status'])
                    
    return df

In [ ]:
def status_calculate_global_totals(df):    
    # gogpt_status_all = df.groupby(['Year', 'Status'])[['Capacity (MW)']].sum().reset_index()
    gogpt_status_all = pd.pivot_table(df, values='Capacity (MW)', index=['Year','Status'], aggfunc='sum', fill_value=0).reset_index()
    gogpt_status_all.insert(0, 'Country', 'all')
    gogpt_status_all = sort_status(gogpt_status_all)
    
    df = pd.concat([gogpt_status_all, df], sort=False)

    return df

In [ ]:
def status_condense(df_arg):
    """
    There are sometimes duplicates, for example if two units in a plant have the exact same capacity and history.
    So before setting the index to Country & Capacity, need to get rid of duplicates.
    """
    df = df_arg.copy()
    df_init = df_arg.copy()  # for test
    
    df = df.dropna(how='all')
    
    df = df.set_index(['Country', 'Capacity (MW)'])
    df = df.stack().reset_index()
        
    df = df.rename(columns={'level_2': 'Year', 0: 'Status'})
    
    df = df.groupby(['Country', 'Year', 'Status'])[['Capacity (MW)']].sum().reset_index()
    df = sort_status(df)

    # print(df.columns) # for db
    status_test_capacity(df, df_init)
    
    return df

In [ ]:
def test_statuses(gogpt_status_uncondensed_merged):
    accepted_statuses = ['announced', 'construction', 'mothballed', 'operating', 'pre-construction', 'shelved']
    for col in ['2021.5', '2022', '2022.5', '2023', '2023.5']:
        ser = gogpt_status_uncondensed_merged[col].dropna()
        unaccepted = ser[~ser.isin(accepted_statuses)]
        if len(unaccepted) > 0:
            print(f"Found unaccepted statuses; len(ser): {len(ser)}")
            print(unaccepted.value_counts())
    # no return

In [ ]:

gogpt_status_uncondensed_merged = status_clean_data(gogpt_status_merged)
test_statuses(gogpt_status_uncondensed_merged)

gogpt_status = status_condense(gogpt_status_uncondensed_merged)
gogpt_status = status_fill_in_missing(gogpt_status)
gogpt_status = status_calculate_global_totals(gogpt_status)

### testing the merged df

In [ ]:
gogpt_status_uncondensed_merged.head()
gogpt_status_uncondensed_merged.sample()

list_of_dfs = [gogpt_status]
country = 'China'
cap = 60.0
for df in list_of_dfs:
    test_df = df[(df['Country'] == country)]
    print(test_df)


### Age & Technology (lower left)
- For bar chart of capacity by age & type
- From the sheet 'Plant type and age'
- same as coal, first make the age variable


In [ ]:
# create age variable
gogpt_age = gogpt_h1_2023[['Country', 'Capacity (MW)', 'Start year', 'Status', 'Technology']].copy()

# TODO filter for operating - double check GCPT too
print(gogpt_age.shape[0])
gogpt_age = gogpt_age[gogpt_age['Status'] == 'operating'] 
print(gogpt_age.shape[0])

gogpt_age['Capacity (MW)'] = pd.to_numeric(gogpt_age['Capacity (MW)'])

gogpt_age = gogpt_age.dropna(subset= ['Start year']) #, 'Technology']
print(gogpt_age.shape[0])

gogpt_age = gogpt_age.drop(gogpt_age[gogpt_age['Start year'] == 'not found'].index) 
print(gogpt_age.shape[0])

gogpt_age['Technology'] = gogpt_age['Technology'].fillna('not found')

plant_ages = []
for value in gogpt_age['Start year']:
    if '-' in str(value):
        first_yr = int(value.split('-')[0].strip())
        sec_yr = int(value.split('-')[1].strip())
        if sec_yr < first_yr:
            print(f"whooops {sec_yr} is less than {first_yr}")
            plant_age = 'not valid'
            plant_ages.append(plant_age)
        elif first_yr >= max_year:
            plant_age = 0
            plant_ages.append(plant_age)
        else:
            plant_age = max_year - int(first_yr)
            plant_ages.append(plant_age)

    elif value >= max_year:
        plant_age = 0
        plant_ages.append(plant_age)
    else:
       plant_age = max_year - int(value)
       # TODO check if no start year is caught
       plant_ages.append(plant_age)


if len(plant_ages) != gogpt_age.shape[0]:
    print("Something went wrong with logic, missing a plant age for a row.")
else:
    gogpt_age['Plant Age'] = plant_ages
        
        
gogpt_age['Capacity (MW)'] = pd.to_numeric(gogpt_age['Capacity (MW)'])



In [ ]:
def age_read_and_clean(gogpt_age):
    # rename technology to be human friendly 
    gogpt_age['Technology'] = gogpt_age['Technology'].replace(
        'GT', 'Gas Turbine'
    )
    gogpt_age['Technology'] = gogpt_age['Technology'].replace(
        'ST', 'Steam Turbine'
    )
    gogpt_age['Technology'] = gogpt_age['Technology'].replace(
        'CC', 'Combined Cycle'
    )
    gogpt_age['Technology'] = gogpt_age['Technology'].replace(
        'ICCC', 'Internal Combustion Combined Cycle'
    )   
    gogpt_age['Technology'] = gogpt_age['Technology'].replace(
        'ISCC', 'Integrated Solar Combined Cycle'
    ) 
    gogpt_age['Technology'] = gogpt_age['Technology'].replace(
        'IC', 'Internal Combustion'
    ) 
    gogpt_age['Technology'] = gogpt_age['Technology'].replace(
        'AFC', 'Allam-Fetvedt Cycle'
    ) 

    # change not found technology to unknown

    gogpt_age['Technology'] = gogpt_age['Technology'].replace(
        'not found', 'Unknown'
    ) 

    return gogpt_age

gogpt_age = age_read_and_clean(gogpt_age)
gogpt_age

In [ ]:
# testing out new way to group
def age_condense_data(df):
    print(len(df))
    # to fix for future warning that dtype is float64 because nan if not set and then incompatible
    # df["Decade"] = ""
    # bin by decade
    for row in df.index:
        age = df.at[row, 'Plant Age']
        if age < 10:
            df.at[row, 'Decade'] = '0-9 years'
        elif age >= 10 and age < 20:
            df.at[row, 'Decade'] = '10-19 years'
        elif age >= 20 and age < 30:
            df.at[row, 'Decade'] = '20-29 years'
        elif age >= 30 and age < 40:
            df.at[row, 'Decade'] = '30-39 years'
        elif age >= 40 and age < 50:
            df.at[row, 'Decade'] = '40-49 years'
        elif age >= 50:
            df.at[row, 'Decade'] = '50+ years'
        else:
            print("Error!" + f" Issue with age for row {row}: {age}")
    
    return df

gogpt_age = age_condense_data(gogpt_age)


In [ ]:
def age_calculate_global_totals(df):
    
    gogpt_age_all = pd.pivot_table(df, values='Capacity (MW)', index=['Decade', 'Technology'], aggfunc='sum', fill_value=0).reset_index()
    gogpt_age_all.insert(0, 'Country', 'all')

    df = pd.concat([gogpt_age_all, df], sort=False)

    return df

gogpt_age = age_calculate_global_totals(gogpt_age)
gogpt_age


In [ ]:

def sum_age(df):
    table_age = pd.pivot_table(df, values='Capacity (MW)', index=['Decade', 'Technology', 'Country'], aggfunc='sum', fill_value=0).reset_index()

    # unstack, then fill in zeros
    # # TODO understand unstack - pivot a level of the df's indexes if not multi index then returns a series 
    # df = df.set_index(['Country', 'Decade', 'Technology']).unstack()
    # df = df.droplevel(0, axis=1)
    table_age = table_age.set_index(['Country', 'Decade', 'Technology']).unstack()
    table_age = table_age.droplevel(0, axis=1)

    for col in table_age.columns:
        table_age[col] = table_age[col].fillna(0)

    # it names the index
    table_age = table_age.reset_index()
    table_age.columns.name = ''
    
    

    return table_age

gogpt_age = sum_age(gogpt_age)
gogpt_age

In [ ]:
# to test against tracker data without doing long sumif statement to create decades
test = gogpt_age[(gogpt_age['Country']=='all') & (gogpt_age['Decade']=='0-9 years')]
test

In [ ]:

def age_fill_in_missing_decades(df):
    decade_list = [
        '0-9 years',
        '10-19 years',
        '20-29 years',
        '30-39 years',
        '40-49 years',
        '50+ years',
    ]
    for country_sel in gogpt_country_list:
        for decade_sel in decade_list:
            # select where the country and decade match within the existing df
            df_sel = df[(df['Country']==country_sel) & (df['Decade']==decade_sel)]
            # print(df_sel)
            # print(len(df_sel))
            if len(df_sel) == 0:
                print(f'length is zero for this decade and country: {country_sel} {decade_sel}')
                new_df = pd.DataFrame.from_dict({
                        'Country': country_sel,
                        'Decade': decade_sel,
                        'Gas Turbine': [float(0)],
                        'Steam Turbine': [float(0)],
                        'Combined Cycle': [float(0)],
                        'Internal Combustion Combined Cycle': [float(0)],
                        'Integrated Solar Combined Cycle': [float(0)],
                        # 'Ultra-Integrated Solar Combined Cycle': [float(0)],
                        'Allam-Fetvedt Cycle': [float(0)],
                        'Internal Combustion': [float(0)],
                        'Unknown': [float(0)],
                    }, orient='columns')
                df = pd.concat([df, new_df], sort=False)

            elif len(df_sel) == 1:

                pass

            elif len(df_sel) > 1:
                print("Error!"+ f'{country_sel} and {decade_sel}')

            else:
                print("Error! (of another kind)")

    return df

gogpt_age = age_fill_in_missing_decades(gogpt_age)

### Additions 
- data for bar chart additions (lower-right)
- sheet '2000-2022' (for example)
- need country year capacity status

In [ ]:
# create add df 
gogpt_add = gogpt_h1_2023[['Country', 'Capacity (MW)', 'Start year', 'Status']].copy()
print(gogpt_add.shape)
gogpt_add = gogpt_add.dropna(subset=['Start year'])
gogpt_add = gogpt_add.drop(gogpt_add[gogpt_add['Start year'] == 'not found'].index) 
print(gogpt_add.shape)



In [ ]:

# handle all year ranges (needed for older data but this graph doesn't use older data)
for row in gogpt_add.index:
    year = gogpt_add.at[row, 'Start year']
    if '-' in str(year):
        first_yr = int(year.split('-')[0].strip())
        gogpt_add.at[row, 'Start year'] = first_yr
    elif ',' in str(year):
        first_yr = int(year.split(',')[0].strip())
        gogpt_add.at[row, 'Start year'] = first_yr
    elif ';'in str(year):
        first_yr = int(year.split(';')[0].strip())
        gogpt_add.at[row, 'Start year'] = first_yr


# replace 'before 1992' with 1992
gogpt_add = gogpt_add.drop(gogpt_add[gogpt_add['Start year'] == 'before 1992'].index)

# remove any years before 2000 NOTE: this is why some sums are different from excel 
gogpt_add = gogpt_add[gogpt_add['Start year'] >= 2000]



In [ ]:
# check that it did it's thing
for value in gogpt_add['Start year']:
    if '-' in str(value):
        print(value)

In [ ]:
# here we just want to drop any rows that don't have status == operating
gogpt_add = gogpt_add[gogpt_add['Status']=='operating']


In [ ]:
gogpt_add = gogpt_add.groupby(['Country','Start year', 'Status']).sum().reset_index()


In [ ]:
def add_unstack(df):
    df = df.set_index(['Country', 'Start year', 'Status'])
    # rearranges / pivots data of df
    # df = df.unstack(-1)
    df = df.unstack()

    # removes first row since: 1 or ‘columns’: remove level(s) in row.
    df = df.droplevel(0, axis=1)
    print(df.head())
    df = df.reset_index()
    df = df.rename(columns={
        'operating': 'Added (MW)', 
        'Start year': 'Year'
    })
        
    return df

In [ ]:
# add missing countries
def add_missing_countries(df):
    # add any missing countries, to make sure that all countries in gcpt_country_list are in gcpt_map
    # missing_countries = [x for x in gcpt_country_list if x not in gcpt_add['Country'].tolist()]
    missing_countries = [x for x in gogpt_country_list if x not in gogpt_add['Country'].tolist()]
    print(missing_countries)

    print(f"Show any countries missing (which will be added below): {missing_countries}")
    
    for year in range(int(gogpt_add['Year'].min()), 
        int(max_year)+1):

        new_df = pd.DataFrame.from_dict({
                'Country': missing_countries,
                'Year': year,
                'Added (MW)': [float(0)]*len(missing_countries),
                # 'Status': 'operating',
            }, orient='columns')
        print(f'this is new df: {new_df}')
        df = pd.concat([df, new_df], sort=False)
    return df


In [ ]:

def fill_in_missing_years(df):

    for country_sel in gogpt_country_list:
        for year in range(int(gogpt_add['Year'].min()), 
            int(max_year)+1):

            df_sel = df[(df['Country']==country_sel) & (df['Year']==year)]
            if len(df_sel) == 0:
                new_df = pd.DataFrame.from_dict({
                        'Country': country_sel,
                        'Year': year,
                        'Added (MW)': [float(0)],
                    }, orient='columns')
                df = pd.concat([df, new_df], sort=False)
    return df

In [ ]:
# then groupby capacities by year and country
def add_calculate_global_totals(df):
    gogpt_add_all = df.groupby(['Year'])[['Added (MW)']].sum().reset_index()
    gogpt_add_all.insert(0, 'Country', 'all')
    df = pd.concat([gogpt_add_all, df], sort=False)
    
    return df

In [ ]:
gogpt_add = add_unstack(gogpt_add)
gogpt_add = add_missing_countries(gogpt_add)
gogpt_add = add_calculate_global_totals(gogpt_add)
gogpt_add = fill_in_missing_years(gogpt_add)


In [ ]:
# test against excel file 
gogpt_add[(gogpt_add['Country']=='all')&(gogpt_add['Year']==2010)]

## Save all to files

In [ ]:
# export to various sheets of one Excel file by creating ExcelWriter object
template_name = gogpt_dash_file_h1_2023.split('.xlsx')[0]
save_timestamp = time.strftime('%Y-%m-%d_%H%M', time.localtime())
file_name = f'{template_name} - processed for Dash {save_timestamp}.xlsx'
with pd.ExcelWriter(gogpt_dash_path + file_name) as writer:
    gogpt_map.to_excel(writer, sheet_name='map', index=False)  
    gogpt_status.to_excel(writer, sheet_name='status', index=False)
    gogpt_age.to_excel(writer, sheet_name='age', index=False)
    gogpt_add.to_excel(writer, sheet_name='additions', index=False)
    print(f"Saved to file: {file_name}")